In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [12]:
DATA_DIR = './data/'
CANCER_DATA = 'cancer_incd_rate_2016_2020.csv'
AQI_DATA = lambda year: f'annual_aqi_by_county_{year}.csv'

In [15]:
def get_cancer_data():
    df = pd.read_csv(DATA_DIR + CANCER_DATA, skiprows=8)
    return df
get_cancer_data()

,County,FIPS,"Age-Adjusted Incidence Rate([rate note]) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,CI*Rank([rank note]),Lower CI (CI*Rank),Upper CI (CI*Rank),Average Annual Count,Recent Trend,Recent 5-Year Trend ([trend note]) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1
0,US (SEER+NPCR)(1),0.0,442.3,442,442.6,N/A,N/A,N/A,1698328,stable,-0.3,-0.6,0.1
1,"Union County, Florida(6)",12125.0,1237.4,1165.6,1312.8,N/A,1,1,237,stable,0.6,-0.5,1.9
2,"Palo Alto County, Iowa(7)",19147.0,658.1,591.1,731.1,N/A,1,6,82,rising,4.8,0.2,15.4
3,"Treasure County, Montana(6)",30103.0,652.2,401,1007.4,N/A,1,55,7,stable,-1.1,-5.6,3.3
4,"Polk County, Texas(7)",48373.0,633.6,604.6,663.7,N/A,1,4,425,rising,2.2,1.2,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3161,7 Source: SEER November 2022 submission.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3162,8 Source: Incidence data provided by the SEER ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3163,Data not available [http://statecancerprofiles...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3164,Data for the United States does not include da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# compile aqi data into 5 year avaerage
def compile_aqi_data():
    annual_aqi_2016_2020 = pd.DataFrame()
    for year in range(2016, 2021):
        aqi = pd.read_csv(DATA_DIR + AQI_DATA(year))
        pd.concat(annual_aqi_2016_2020, aqi)
    
    
    return annual_aqi_2016_2020

compile_aqi_data()

     State   County  Year  Days with AQI  Good Days  Moderate Days  \
0  Alabama  Baldwin  2016            279        247             32   
1  Alabama     Clay  2016            116        109              7   
2  Alabama  Colbert  2016            282        258             23   
3  Alabama   DeKalb  2016            348        304             43   
4  Alabama   Elmore  2016            117        107             10   

   Unhealthy for Sensitive Groups Days  Unhealthy Days  Very Unhealthy Days  \
0                                    0               0                    0   
1                                    0               0                    0   
2                                    1               0                    0   
3                                    1               0                    0   
4                                    0               0                    0   

   Hazardous Days  Max AQI  90th Percentile AQI  Median AQI  Days CO  \
0               0       87      